## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)
application_df.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
unique_value_counts = application_df.nunique()
print(unique_value_counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_type_counts)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 10
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 1000
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()


# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
df_encoded = pd.get_dummies(application_df)


In [10]:
# Split our preprocessed data into our features and target arrays
X = df_encoded.drop('IS_SUCCESSFUL', axis=1)
y = df_encoded['IS_SUCCESSFUL']
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Optimize Model

In [24]:
# Try#1: increasing Epochs
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = X_train_scaled.shape[1]

hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn_model = tf.keras.models.Sequential()

output_nodes = 1

# First hidden layer
nn_model.add(tf.keras.layers.Dense(hidden_nodes_layer1, input_dim=input_features, activation='relu'))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(hidden_nodes_layer2, activation='relu'))

# Output layer
nn_model.add(tf.keras.layers.Dense(output_nodes, activation='sigmoid'))

# Check the structure of the model
nn_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 80)                3840      
                                                                 
 dense_12 (Dense)            (None, 30)                2430      
                                                                 
 dense_13 (Dense)            (None, 1)                 31        
                                                                 
Total params: 6,301
Trainable params: 6,301
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Compile the model
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [26]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5739 - accuracy: 0.7201
Epoch 2/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5561 - accuracy: 0.7318
Epoch 3/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5528 - accuracy: 0.7310
Epoch 4/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5512 - accuracy: 0.7319
Epoch 5/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5500 - accuracy: 0.7327
Epoch 6/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5487 - accuracy: 0.7333
Epoch 7/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5479 - accuracy: 0.7334
Epoch 8/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5478 - accuracy: 0.7342
Epoch 9/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5472 - accuracy: 0.7343
Epoch 10/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5454 - accura

In [27]:
# Evaluate the model using the test data
nn_model_loss, nn_model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {nn_model_loss}, Accuracy: {nn_model_accuracy}")

268/268 - 0s - loss: 0.5625 - accuracy: 0.7241 - 484ms/epoch - 2ms/step
Loss: 0.5624664425849915, Accuracy: 0.7240816354751587


In [28]:
# Try#2: Adding one more layer
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = X_train_scaled.shape[1]

hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn_model = tf.keras.models.Sequential()

output_nodes = 1

# First hidden layer
nn_model.add(tf.keras.layers.Dense(hidden_nodes_layer1, input_dim=input_features, activation='relu'))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(hidden_nodes_layer2, activation='relu'))

# Third hidden layer
nn_model.add(tf.keras.layers.Dense(hidden_nodes_layer3, activation='relu'))


# Output layer
nn_model.add(tf.keras.layers.Dense(output_nodes, activation='sigmoid'))

# Check the structure of the model
nn_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 80)                3840      
                                                                 
 dense_15 (Dense)            (None, 30)                2430      
                                                                 
 dense_16 (Dense)            (None, 10)                310       
                                                                 
 dense_17 (Dense)            (None, 1)                 11        
                                                                 
Total params: 6,591
Trainable params: 6,591
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Compile the model
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5771 - accuracy: 0.7134
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5543 - accuracy: 0.7284
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5512 - accuracy: 0.7315
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5501 - accuracy: 0.7334
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5483 - accuracy: 0.7346
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5476 - accuracy: 0.7348
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5472 - accuracy: 0.7341
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5465 - accuracy: 0.7357
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5453 - accuracy: 0.7365
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5448 - accura

In [31]:
# Evaluate the model using the test data
nn_model_loss, nn_model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {nn_model_loss}, Accuracy: {nn_model_accuracy}")

268/268 - 1s - loss: 0.5575 - accuracy: 0.7264 - 518ms/epoch - 2ms/step
Loss: 0.557498037815094, Accuracy: 0.7264139652252197


In [32]:
# Try#3: changing different actication
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = X_train_scaled.shape[1]

hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn_model = tf.keras.models.Sequential()

output_nodes = 1

# First hidden layer
nn_model.add(tf.keras.layers.Dense(hidden_nodes_layer1, input_dim=input_features, activation='tanh'))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(hidden_nodes_layer2, activation='tanh'))

# Output layer
nn_model.add(tf.keras.layers.Dense(output_nodes, activation='sigmoid'))

# Check the structure of the model
nn_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 80)                3840      
                                                                 
 dense_19 (Dense)            (None, 30)                2430      
                                                                 
 dense_20 (Dense)            (None, 1)                 31        
                                                                 
Total params: 6,301
Trainable params: 6,301
Non-trainable params: 0
_________________________________________________________________


In [33]:
# Compile the model
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [34]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5706 - accuracy: 0.7227
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5558 - accuracy: 0.7302
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5525 - accuracy: 0.7322
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5507 - accuracy: 0.7318
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5491 - accuracy: 0.7332
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5477 - accuracy: 0.7337
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5472 - accuracy: 0.7347
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5459 - accuracy: 0.7346
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5456 - accuracy: 0.7352
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5452 - accura

In [35]:
# Evaluate the model using the test data
nn_model_loss, nn_model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {nn_model_loss}, Accuracy: {nn_model_accuracy}")

268/268 - 0s - loss: 0.5551 - accuracy: 0.7249 - 462ms/epoch - 2ms/step
Loss: 0.5551292300224304, Accuracy: 0.7248979806900024


In [36]:
# Export our model to HDF5 file
nn_model.save('model.h5')